In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

densenet_path = '../input/dense_test_corr.h5'
BATCHES_COUNT = 12091
TEST_SIZE = 1768182

In [2]:
%%time
store = pd.HDFStore(densenet_path)
densenet_keys = store.keys()
print('{} tables in storage'.format(len(densenet_keys)))

12091 tables in storage
CPU times: user 38.3 s, sys: 768 ms, total: 39 s
Wall time: 43.9 s


# Усреднить только денснет

In [10]:
MULTIPLY_FOR_MAX = 1

dense_preds = []
cur_id = None
sums = pd.DataFrame()
cur_sum = None

for key in tqdm(densenet_keys): 
    table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    for i in range(table.shape[0]):
        product_id = table.pr_id.iloc[i]
        new_sum = table[i : i + +1].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        if cur_id == product_id:
            assert cur_sum is not None
            sums.append(new_sum)
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            print(cur_sum)
            for i in range(MULTIPLY_FOR_MAX):
                cur_sum += sums.max()
            print(cur_sum)
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            dense_preds.append((int(cur_id), cat))
            
        cur_id = product_id
        cur_sum = new_sum
        sums.drop(sums.index, inplace=True)
        sums.append(new_sum, ignore_index=True)
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    dense_preds.append((int(cur_id), cat))
            
dense_preds_df = pd.DataFrame(data=dense_preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(dense_preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(dense_preds_df.shape[0] - TEST_SIZE,
                                                float(dense_preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
dense_no_dupls = dense_preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape:', dense_no_dupls.shape)

dense_no_dupls.to_csv('../submit/dense_EXPERIMENT.csv', index=False)

  0%|          | 0/12091 [00:00<?, ?it/s]

1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
5241    0.0
5242    0.0
5243    0.0
5244    0.0
5245    0.0
5246    0.0
5247    0.0
5248    0.0
5249    0.0
5250    0.0
5251    0.0
5252    0.0
5253    0.0
5254    0.0
5255    0.0
5256    0.0
5257    0.0
5258    0.0
5259    0.0
5260    0.0
5261    0.0
5262    0.0
5263    0.0
5264    0.0
5265    0.0
5266    0.0
5267    0.0
5268    0.0
5269    0.0
5270    0.0
Length: 5270, dtype: float16
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     N

ValueError: cannot convert float NaN to integer

# Усреднить денснет с резнетом

In [5]:
%%time
res_store = pd.HDFStore('../input/predict_probs_resnet50.h5')
res_keys = res_store.keys()
print('{} tables in storage'.format(len(res_keys)))

0 tables in storage
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.55 ms


In [ ]:
ave_preds = []
cur_id = None
cur_sum = None

for key in tqdm(densenet_keys): 
    res_table = res_store.select(key)
    dens_table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert res_table.shape[0] % 10 == 0
    assert dens_table.shape[0] == res_table.shape[0]
    for i in range(int(res_table.shape[0] / 10)):
        product_id1 = res_table.pr_id.iloc[10*i]
        product_id2 = dens_table.pr_id.iloc[10 * i]
        assert product_id1 == product_id2
        new_sum1 = res_table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        new_sum2 = dens_table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        new_sum = new_sum1 + new_sum2
        
        if cur_id == product_id1:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            ave_preds.append((int(cur_id), cat))
            
        cur_id = product_id1
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    ave_preds.append((int(cur_id), cat))
            
ave_preds_df = pd.DataFrame(data=ave_preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(ave_preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(ave_preds_df.shape[0] - TEST_SIZE,
                                                float(ave_preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
ave_no_dupls = ave_preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape:', ave_no_dupls.shape)

ave_no_dupls.to_csv('../submit/dense_resnet_ave.csv', index=False)